## Working, but ugly :(

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import pandas as pd
import metal
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
from dataset import BERTDataset

In [4]:
model = 'bert-base-uncased' # also try bert-base-multilingual-cased (recommended)
src_path = os.path.join(os.environ['GLUEDATA'], 'SST-2/{}.tsv')
dataloaders = {}
for split in ['train','dev']:
    dataset = BERTDataset(
        src_path.format(split),
        sent1_idx=0,
        label_idx=1,
        skip_rows=1,
        label_fn=lambda x: int(x)+1 # labels are scores [1, 2] (multiclass with cardinality k)
    )

    dataloaders[split] = dataset.get_dataloader(batch_size=8)

100%|██████████| 872/872 [00:00<00:00, 2807.14it/s]


In [5]:
import torch.nn as nn
from metal.end_model import EndModel

class BertEncoder(nn.Module):
    def __init__(self):
        super(BertEncoder, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')

    def forward(self, data):
        tokens, segments, mask = data
        # TODO: check if we should return all layers or just last hidden representation 
        _, hidden_layer = self.bert_model(tokens, segments, mask)
        return hidden_layer

class SSTHead(EndModel):     
    def __init__(self, output_dims, **kwargs):
        super(SSTHead, self).__init__(output_dims, **kwargs)
        # self.criteria = nn.modules.loss.MSELoss()
    """
    def _loss(self, data, Y):
        output = self.forward(data)
        prediction = torch.sigmoid(output)
        return self.criteria(prediction, Y)

    def _get_loss_fn(self):
        # This self.preprocess_Y allows us to not handle preprocessing
        # in a custom dataloader, but decreases speed a bit
        return self._loss
    """

In [6]:
encoder_module = BertEncoder()
end_model = SSTHead(
    [768, 2],
    input_module=encoder_module,
    seed=123,
    device = torch.device("cuda"),
    skip_head=False,
    input_relu=False,
    input_batchnorm=False,
    verbose=False
)

In [ ]:
end_model.train_model(dataloaders['train'], valid_data=dataloaders['dev'],
                      lr=0.01, l2=0.01, 
                      n_epochs=1, checkpoint_metric='train/loss',
                      log_valid_metrics=["train/loss", "valid/loss"],
                      checkpoint_metric_mode='min',
                      verbose=True, progress_bar=True
                    )

Using GPU...


Saving model at iteration 0 with best score 0.663
